# CLARA + HDC MEMORY INTEGRATION

This notebook integrates Hyperdimensional Computing (HDC) memory into Clara's dual-brain architecture.

**Prerequisites:**
- Trained models in Google Drive (`/Lily/models/`)
- `clara-knowledge` (Phi-3 merged)
- `mistral_warmth`, `mistral_playful`, `mistral_encouragement` adapters

**What this adds:**
- HDC Memory System (~200KB footprint)
- Memory-augmented routing
- Interaction history storage
- Personality vectors
- Memory persistence (save/load)

**Architecture:**
```
User Query → HDC Memory Context → Semantic Router → Brain Selection → Response
                    ↑                                                    │
                    └────────────── Store Interaction ──────────────────┘
```

## Cell 1: Setup & Installation

In [ ]:
# ============================================================
# SETUP
# ============================================================

!pip install -q transformers accelerate bitsandbytes
!pip install -q peft sentence-transformers

from google.colab import drive
drive.mount('/content/drive')

import torch
import os

print("=" * 60)
print("SETUP COMPLETE")
print("=" * 60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Cell 2: Check Existing Models (Safety Check)

In [ ]:
# ============================================================
# SAFETY CHECK - Verify existing models without overwriting
# ============================================================

MODELS_DIR = "/content/drive/MyDrive/Lily/models"
MEMORY_DIR = "/content/drive/MyDrive/Lily/memory"

print("=" * 60)
print("MODEL VERIFICATION")
print("=" * 60)

# Required models
required_models = {
    "clara-knowledge": "Phi-3 merged knowledge brain",
    "mistral_warmth": "Personality adapter (warmth)",
    "mistral_playful": "Personality adapter (playful)", 
    "mistral_encouragement": "Personality adapter (encouragement)",
}

all_present = True
model_status = {}

print("\n📁 Checking models in:", MODELS_DIR)
print("-" * 60)

for model_name, description in required_models.items():
    path = os.path.join(MODELS_DIR, model_name)
    
    # Check for key files that indicate a valid model
    if model_name == "clara-knowledge":
        # Merged model should have config.json
        check_file = os.path.join(path, "config.json")
    else:
        # LoRA adapters have adapter_config.json
        check_file = os.path.join(path, "adapter_config.json")
    
    exists = os.path.exists(check_file)
    model_status[model_name] = exists
    
    if exists:
        # Get size
        total_size = sum(
            os.path.getsize(os.path.join(path, f))
            for f in os.listdir(path)
            if os.path.isfile(os.path.join(path, f))
        ) / 1e9
        print(f"  ✅ {model_name:<25} ({total_size:.2f} GB)")
        print(f"      └─ {description}")
    else:
        print(f"  ❌ {model_name:<25} MISSING!")
        print(f"      └─ {description}")
        all_present = False

print("-" * 60)

if all_present:
    print("\n✅ All required models found!")
    print("   Models will NOT be overwritten.")
else:
    print("\n⚠️  Some models are missing!")
    print("   Please run your training notebook first.")
    print("   This notebook requires pre-trained models.")

# Check/create memory directory
print("\n📁 Memory directory:", MEMORY_DIR)
if not os.path.exists(MEMORY_DIR):
    os.makedirs(MEMORY_DIR)
    print("   Created new memory directory")
else:
    # Check for existing memory files
    memory_files = [f for f in os.listdir(MEMORY_DIR) if f.endswith('.json')]
    if memory_files:
        print(f"   Found {len(memory_files)} existing memory file(s):")
        for mf in memory_files:
            size = os.path.getsize(os.path.join(MEMORY_DIR, mf)) / 1024
            print(f"      └─ {mf} ({size:.1f} KB)")
    else:
        print("   No existing memory files (fresh start)")

## Cell 3: Load Semantic Router (Embedder)

In [ ]:
# ============================================================
# SEMANTIC ROUTER - Load embedder and domain descriptions
# ============================================================

from sentence_transformers import SentenceTransformer
import numpy as np

print("=" * 60)
print("LOADING SEMANTIC ROUTER")
print("=" * 60)

print("\n1. Loading embedding model...")
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("   ✅ all-MiniLM-L6-v2 loaded")

# Domain descriptions (refined from your notebook)
DOMAIN_DESCRIPTIONS = {
    "medical": """
        symptoms diagnosis treatment disease illness pain fever infection
        headache nauseous dizzy blood pressure heart lungs brain body
        doctor hospital medicine medication prescription surgery vaccine
        virus bacteria immune system allergies chronic acute patient health
        tired fatigue exhausted sleep insomnia rash swollen sore throat
        cough breathing chest stomach ache injury wound bleeding
    """,
    
    "coding": """
        programming code software python javascript java function method
        variable array list dictionary tuple loop error exception bug debug
        API database SQL server backend frontend algorithm data structure
        class object inheritance compile runtime syntax IndexError TypeError
        iterate parse return import library framework git repository
        crash deploy package module script terminal command line
        Flask Django React Node npm pip install developer
    """,
    
    "teaching": """
        explain how does work basics fundamentals introduction tutorial
        step by step concept theory lesson learn teach education student
        example analogy walk through ELI5 for dummies guide overview
        what is the difference between simple explanation textbook
        homework assignment class course study
    """,
    
    "quantum": """
        quantum physics qubit superposition entanglement wave function
        particle measurement collapse observer Schrodinger Heisenberg
        quantum computer quantum gate Hadamard CNOT quantum circuit
        coherence decoherence probability amplitude interference
        quantum mechanics quantum state Planck photon electron spin
        two places at once two states uncertainty principle
        parallel universes both alive and dead particle wave duality
    """,
    
    "personality": """
        feeling emotion mood happy sad angry anxious worried stressed
        excited nervous scared lonely depressed overwhelmed frustrated
        relationship friend family love support talk vent chat
        my day rough tough great amazing terrible celebrate
        broke up promotion new job interview date
        grateful appreciate thankful thanks thank you
        hey hi hello how are you good morning good night
        nobody understands me moving away miss you
        sorry to hear congratulations best friend
        take care see you later nice to meet you
    """,
}

print("\n2. Computing domain embeddings...")
domain_embeddings = {}
for domain, description in DOMAIN_DESCRIPTIONS.items():
    clean_desc = " ".join(description.split())
    domain_embeddings[domain] = embedder.encode(clean_desc)
    print(f"   ✅ {domain}")

print("\n✅ Semantic router ready")

## Cell 4: HDC Memory System

In [ ]:
# ============================================================
# HDC MEMORY SYSTEM
# ============================================================

from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
import time
import json

print("=" * 60)
print("HDC MEMORY SYSTEM")
print("=" * 60)

@dataclass
class Memory:
    """Single memory unit"""
    text: str
    timestamp: float
    memory_type: str  # 'interaction', 'preference', 'fact'
    importance: float = 0.5
    domain: str = "general"


class ClaraHDCMemory:
    """
    Hyperdimensional Computing Memory System for Clara
    
    Features:
    - Reuses existing embedder (no additional model loading)
    - 10,000-dimension bipolar hypervectors
    - Structured binding for compositional memories
    - Personality vectors influence routing
    - ~200KB footprint (excluding embedder)
    
    HDC Operations:
    - Bind (⊗): Element-wise multiplication - creates associations
    - Bundle (+): Element-wise addition + sign - superposition
    - Similarity: Cosine similarity for retrieval
    """
    
    def __init__(self, embedder, dim: int = 10000, seed: int = 42):
        self.dim = dim
        self.embedder = embedder  # Reuse existing embedder
        self.rng = np.random.RandomState(seed)
        
        # Random projection matrix: 384-dim → 10K-dim
        print("   Initializing projection matrix...")
        self.projection = self.rng.randn(384, dim).astype(np.float32)
        self.projection /= np.linalg.norm(self.projection, axis=1, keepdims=True)
        
        # Memory stores
        self.memories: List[Tuple[np.ndarray, Memory]] = []
        self.memory_bundle = np.zeros(dim, dtype=np.float32)
        
        # Symbol library for structured binding
        print("   Building symbol library...")
        self.symbols: Dict[str, np.ndarray] = {}
        self._init_symbols()
        
        # Personality vectors
        print("   Encoding personality vectors...")
        self.personality = self._init_personality()
        
        print(f"   ✅ HDC Memory initialized (dim={dim})")
    
    def _init_symbols(self):
        """Create base symbol vocabulary"""
        base_symbols = [
            # Roles
            "ROLE_USER", "ROLE_CLARA", "ROLE_TOPIC", "ROLE_OUTCOME",
            # Actions  
            "ASKED", "ANSWERED", "STRUGGLED", "SUCCEEDED", "PREFERRED",
            # Domains (match router)
            "MEDICAL", "CODING", "TEACHING", "QUANTUM", "PERSONALITY",
            # Outcomes
            "HELPFUL", "CONFUSED", "SATISFIED", "FRUSTRATED",
            # Time markers
            "RECENT", "TODAY", "THIS_SESSION"
        ]
        for s in base_symbols:
            self.symbols[s] = self._random_hv()
    
    def _init_personality(self) -> Dict[str, np.ndarray]:
        """Encode Clara's personality traits as hypervectors"""
        traits = {
            'warmth': 0.85,
            'curiosity': 0.75,
            'patience': 0.90,
            'encouragement': 0.80,
        }
        personality = {}
        for trait, strength in traits.items():
            base_hv = self._random_hv()
            personality[trait] = (base_hv * strength).astype(np.float32)
        
        # Composite personality vector
        all_traits = list(personality.values())
        personality['composite'] = np.sign(
            np.sum(all_traits, axis=0)
        ).astype(np.float32)
        
        return personality
    
    def _random_hv(self) -> np.ndarray:
        """Generate random bipolar hypervector {-1, +1}"""
        return self.rng.choice([-1, 1], size=self.dim).astype(np.float32)
    
    def _text_to_hv(self, text: str) -> np.ndarray:
        """Convert text to hypervector using existing embedder"""
        embedding = self.embedder.encode(text)
        hv = embedding @ self.projection
        return np.sign(hv).astype(np.float32)
    
    def _get_symbol(self, name: str) -> np.ndarray:
        """Get or create symbol"""
        name_upper = name.upper()
        if name_upper not in self.symbols:
            self.symbols[name_upper] = self._random_hv()
        return self.symbols[name_upper]
    
    # === HDC Operations ===
    
    def bind(self, hv1: np.ndarray, hv2: np.ndarray) -> np.ndarray:
        """Bind two hypervectors (⊗) - creates association"""
        return hv1 * hv2
    
    def bundle(self, hvs: List[np.ndarray]) -> np.ndarray:
        """Bundle hypervectors (+) - superposition"""
        if not hvs:
            return np.zeros(self.dim, dtype=np.float32)
        return np.sign(np.sum(hvs, axis=0)).astype(np.float32)
    
    def similarity(self, hv1: np.ndarray, hv2: np.ndarray) -> float:
        """Cosine similarity between hypervectors"""
        n1, n2 = np.linalg.norm(hv1), np.linalg.norm(hv2)
        if n1 == 0 or n2 == 0:
            return 0.0
        return float(np.dot(hv1, hv2) / (n1 * n2))
    
    # === Memory Operations ===
    
    def store(self, text: str, memory_type: str = "interaction",
              importance: float = 0.5, domain: str = "general",
              **bindings) -> None:
        """
        Store a memory with optional structured bindings
        
        Args:
            text: The memory content
            memory_type: 'interaction', 'preference', 'fact'
            importance: 0.0-1.0 importance score
            domain: 'medical', 'coding', 'teaching', 'quantum', 'personality'
            **bindings: Structured role-filler pairs
            
        Example:
            memory.store("User asked about async Python",
                        memory_type="interaction",
                        domain="coding",
                        topic="ASYNC_PYTHON", outcome="ANSWERED")
        """
        # Create semantic hypervector
        text_hv = self._text_to_hv(text)
        
        # Add domain binding
        domain_hv = self._get_symbol(domain.upper())
        text_hv = self.bind(text_hv, domain_hv)
        
        # Add structural bindings if provided
        if bindings:
            bound_parts = []
            for role, filler in bindings.items():
                role_hv = self._get_symbol(f"ROLE_{role.upper()}")
                filler_hv = self._get_symbol(str(filler).upper())
                bound_parts.append(self.bind(role_hv, filler_hv))
            if bound_parts:
                structure_hv = self.bundle(bound_parts)
                text_hv = self.bundle([text_hv, structure_hv])
        
        memory = Memory(
            text=text,
            timestamp=time.time(),
            memory_type=memory_type,
            importance=importance,
            domain=domain
        )
        
        self.memories.append((text_hv, memory))
        
        # Update bundled representation
        self.memory_bundle = self.bundle([self.memory_bundle, text_hv])
    
    def recall(self, query: str, top_k: int = 3,
               domain_filter: Optional[str] = None) -> List[Tuple[Memory, float]]:
        """
        Retrieve memories similar to query
        
        Args:
            query: Search query
            top_k: Number of results
            domain_filter: Optional domain to filter by
            
        Returns:
            List of (Memory, similarity_score) tuples
        """
        if not self.memories:
            return []
        
        query_hv = self._text_to_hv(query)
        
        results = []
        for hv, memory in self.memories:
            # Apply domain filter if specified
            if domain_filter and memory.domain != domain_filter:
                continue
            
            sim = self.similarity(query_hv, hv)
            
            # Weight by importance and recency
            age_hours = (time.time() - memory.timestamp) / 3600
            recency = 1.0 / (1.0 + age_hours / 24)
            weighted = sim * (0.7 + 0.3 * memory.importance) * (0.8 + 0.2 * recency)
            
            results.append((memory, weighted))
        
        results.sort(key=lambda x: x[1], reverse=True)
        return results[:top_k]
    
    def get_context_for_routing(self, query: str) -> np.ndarray:
        """
        Get memory-augmented context for routing decisions
        
        Combines:
        - Query semantics
        - Relevant memory context
        - Personality alignment
        """
        query_hv = self._text_to_hv(query)
        
        # Get relevant memory context
        relevant = self.recall(query, top_k=3)
        if relevant:
            memory_hvs = [self._text_to_hv(m.text) for m, _ in relevant]
            memory_context = self.bundle(memory_hvs)
        else:
            memory_context = np.zeros(self.dim, dtype=np.float32)
        
        # Combine: query + memory + personality
        routing_hv = self.bundle([
            query_hv,
            memory_context * 0.3,
            self.personality['composite'] * 0.2
        ])
        
        return routing_hv
    
    def get_context_string(self, query: str, max_memories: int = 2) -> str:
        """
        Get memory context as string for prompt injection
        
        Returns formatted string to include in LLM prompt
        """
        relevant = self.recall(query, top_k=max_memories)
        if not relevant:
            return ""
        
        # Only include memories above threshold
        good_memories = [(m, s) for m, s in relevant if s > 0.25]
        if not good_memories:
            return ""
        
        context_parts = ["[Previous context:"]
        for mem, score in good_memories:
            context_parts.append(f"- {mem.text[:80]}")
        context_parts.append("]")
        
        return "\n".join(context_parts)
    
    def get_domain_history(self, domain: str) -> List[Memory]:
        """Get all memories for a specific domain"""
        return [m for _, m in self.memories if m.domain == domain]
    
    def size_bytes(self) -> int:
        """Memory footprint (excluding embedder)"""
        bundle_size = self.memory_bundle.nbytes
        memories_size = sum(hv.nbytes for hv, _ in self.memories)
        symbols_size = sum(hv.nbytes for hv in self.symbols.values())
        personality_size = sum(hv.nbytes for hv in self.personality.values())
        projection_size = self.projection.nbytes
        return bundle_size + memories_size + symbols_size + personality_size + projection_size
    
    def stats(self) -> Dict:
        """Get memory statistics"""
        domain_counts = {}
        for _, m in self.memories:
            domain_counts[m.domain] = domain_counts.get(m.domain, 0) + 1
        
        return {
            "total_memories": len(self.memories),
            "by_domain": domain_counts,
            "symbols": len(self.symbols),
            "size_kb": self.size_bytes() / 1024
        }
    
    # === Persistence ===
    
    def save(self, path: str):
        """Save memory state to file"""
        data = {
            'version': '1.0',
            'dim': self.dim,
            'memories': [
                {
                    'hv': hv.tolist(),
                    'text': m.text,
                    'timestamp': m.timestamp,
                    'memory_type': m.memory_type,
                    'importance': m.importance,
                    'domain': m.domain
                }
                for hv, m in self.memories
            ],
            'symbols': {k: v.tolist() for k, v in self.symbols.items()},
            'memory_bundle': self.memory_bundle.tolist(),
            'projection': self.projection.tolist()
        }
        
        with open(path, 'w') as f:
            json.dump(data, f)
        
        print(f"✅ Saved {len(self.memories)} memories to {path}")
        print(f"   Size: {os.path.getsize(path) / 1024:.1f} KB")
    
    def load(self, path: str):
        """Load memory state from file"""
        if not os.path.exists(path):
            print(f"⚠️ No memory file found at {path}")
            return False
        
        with open(path) as f:
            data = json.load(f)
        
        # Validate version and dimension
        if data.get('dim') != self.dim:
            print(f"⚠️ Dimension mismatch: file has {data.get('dim')}, expected {self.dim}")
            return False
        
        # Load memories
        self.memories = [
            (
                np.array(m['hv'], dtype=np.float32),
                Memory(
                    text=m['text'],
                    timestamp=m['timestamp'],
                    memory_type=m['memory_type'],
                    importance=m['importance'],
                    domain=m.get('domain', 'general')
                )
            )
            for m in data['memories']
        ]
        
        # Load symbols
        self.symbols = {
            k: np.array(v, dtype=np.float32)
            for k, v in data['symbols'].items()
        }
        
        # Load bundle
        self.memory_bundle = np.array(data['memory_bundle'], dtype=np.float32)
        
        # Load projection if present (for consistency)
        if 'projection' in data:
            self.projection = np.array(data['projection'], dtype=np.float32)
        
        print(f"✅ Loaded {len(self.memories)} memories from {path}")
        return True


# ============================================================
# Initialize HDC Memory
# ============================================================
print("\nInitializing Clara's HDC Memory...")
clara_memory = ClaraHDCMemory(embedder=embedder, dim=10000)

# Try to load existing memory
MEMORY_FILE = os.path.join(MEMORY_DIR, "clara_memory.json")
if os.path.exists(MEMORY_FILE):
    print(f"\nFound existing memory file, loading...")
    clara_memory.load(MEMORY_FILE)
else:
    print(f"\nNo existing memory file - starting fresh")

print(f"\n📊 Memory Stats:")
stats = clara_memory.stats()
print(f"   Total memories: {stats['total_memories']}")
print(f"   Symbols: {stats['symbols']}")
print(f"   Size: {stats['size_kb']:.1f} KB")

## Cell 5: Smart Router with Memory

In [ ]:
# ============================================================
# SMART ROUTER (with memory integration)
# ============================================================

print("=" * 60)
print("SMART ROUTER")
print("=" * 60)

def smart_route(query: str, use_memory: bool = True, 
                threshold: float = 0.20) -> tuple:
    """
    Route query using semantic similarity + memory context
    
    Args:
        query: User input
        use_memory: Whether to incorporate memory context
        threshold: Minimum confidence threshold
        
    Returns:
        (brain_type, domain, confidence, memory_context_used)
    """
    # Get query embedding
    query_embedding = embedder.encode(query)
    
    # Calculate similarities to each domain
    similarities = {}
    for domain, domain_emb in domain_embeddings.items():
        similarity = np.dot(query_embedding, domain_emb) / (
            np.linalg.norm(query_embedding) * np.linalg.norm(domain_emb)
        )
        similarities[domain] = similarity
    
    # Memory-based boosting
    memory_context_used = False
    if use_memory and len(clara_memory.memories) > 0:
        relevant = clara_memory.recall(query, top_k=2)
        if relevant:
            for mem, score in relevant:
                if score > 0.3 and mem.domain in similarities:
                    # Boost the domain from memory
                    similarities[mem.domain] += 0.05
                    memory_context_used = True
    
    # Sort by similarity
    sorted_domains = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    best_domain, best_conf = sorted_domains[0]
    second_domain, second_conf = sorted_domains[1]
    
    # If top two are very close and one is personality, prefer personality
    # (Clara should be warm/supportive when uncertain)
    if best_conf - second_conf < 0.05:
        if second_domain == "personality":
            best_domain = "personality"
            best_conf = second_conf
    
    # Low confidence fallback
    if best_conf < threshold and best_domain != "personality":
        if similarities["personality"] > 0.15:
            best_domain = "personality"
            best_conf = similarities["personality"]
    
    # Determine brain type
    if best_domain == "personality":
        brain = "personality"
        domain = "warmth"  # Default adapter
    else:
        brain = "knowledge"
        domain = best_domain
    
    return brain, domain, best_conf, memory_context_used


def clean_response(response: str) -> str:
    """Clean up response artifacts"""
    stop_markers = ["### Instruction:", "Instruction:", "\n\n\n", "User:"]
    for marker in stop_markers:
        if marker in response:
            response = response.split(marker)[0].strip()
    return response.strip()


print("✅ Router functions defined")

# Quick test
print("\n📋 Router test:")
test_queries = [
    "How do I read a CSV in Python?",
    "I'm feeling stressed about work",
    "What is quantum entanglement?"
]
for q in test_queries:
    brain, domain, conf, mem_used = smart_route(q)
    mem_icon = "🧠" if mem_used else "  "
    print(f"   {mem_icon} {q[:40]:40} → {brain}/{domain} ({conf:.2f})")

## Cell 6: Load Clara's Brains

In [ ]:
# ============================================================
# LOAD CLARA'S DUAL-BRAIN SYSTEM
# ============================================================

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

print("=" * 60)
print("LOADING CLARA'S BRAINS")
print("=" * 60)

# ============================================================
# KNOWLEDGE BRAIN (Phi-3 merged)
# ============================================================
print("\n1. Loading Knowledge Brain (Phi-3)...")

knowledge_path = os.path.join(MODELS_DIR, "clara-knowledge")

if not os.path.exists(knowledge_path):
    raise FileNotFoundError(f"Knowledge model not found: {knowledge_path}")

knowledge_tokenizer = AutoTokenizer.from_pretrained(
    knowledge_path,
    trust_remote_code=True
)

knowledge_model = AutoModelForCausalLM.from_pretrained(
    knowledge_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
knowledge_model.eval()
print("   ✅ Knowledge brain loaded")

# ============================================================
# PERSONALITY BRAIN (Mistral + LoRA adapters)
# ============================================================
print("\n2. Loading Personality Brain (Mistral + adapters)...")

personality_tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3"
)

personality_base = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Load adapters
adapters_to_load = [
    ("warmth", "mistral_warmth"),
    ("playful", "mistral_playful"),
    ("encouragement", "mistral_encouragement"),
]

# Load first adapter
first_name, first_path = adapters_to_load[0]
personality_model = PeftModel.from_pretrained(
    personality_base,
    os.path.join(MODELS_DIR, first_path),
    adapter_name=first_name
)

# Load remaining adapters
for adapter_name, adapter_path in adapters_to_load[1:]:
    full_path = os.path.join(MODELS_DIR, adapter_path)
    if os.path.exists(full_path):
        personality_model.load_adapter(full_path, adapter_name=adapter_name)
        print(f"   ✅ Loaded adapter: {adapter_name}")
    else:
        print(f"   ⚠️ Adapter not found: {adapter_path}")

personality_model.set_adapter("warmth")  # Default
personality_model.eval()
print("   ✅ Personality brain loaded")

print("\n" + "=" * 60)
print("🧠 CLARA'S BRAINS ARE READY!")
print("=" * 60)

## Cell 7: Clara Main Interface (with Memory)

In [ ]:
# ============================================================
# CLARA - Main Interface with HDC Memory
# ============================================================

print("=" * 60)
print("CLARA - MAIN INTERFACE")
print("=" * 60)

def clara(query: str, verbose: bool = True, 
          use_memory: bool = True,
          store_interaction: bool = True) -> str:
    """
    Clara's main interface - now with memory!
    
    Args:
        query: User input
        verbose: Print routing info
        use_memory: Use memory for context
        store_interaction: Store this interaction in memory
        
    Returns:
        Clara's response
    """
    
    # 1. Route the query
    brain, domain, conf, mem_used = smart_route(query, use_memory=use_memory)
    
    if verbose:
        mem_icon = "🧠" if mem_used else "  "
        print(f"   {mem_icon} Routing: {brain}/{domain} (conf: {conf:.2f})")
    
    # 2. Get memory context for prompt
    memory_context = ""
    if use_memory:
        memory_context = clara_memory.get_context_string(query)
        if memory_context and verbose:
            print(f"   📚 Memory context included")
    
    # 3. Generate response
    if brain == "knowledge":
        # Knowledge brain (Phi-3)
        if memory_context:
            prompt = f"### Instruction:\n{memory_context}\n\nUser question: {query}\n\n### Response:\n"
        else:
            prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        
        inputs = knowledge_tokenizer(prompt, return_tensors="pt").to(knowledge_model.device)
        
        with torch.no_grad():
            outputs = knowledge_model.generate(
                **inputs,
                max_new_tokens=250,
                temperature=0.7,
                do_sample=True,
                pad_token_id=knowledge_tokenizer.eos_token_id,
                use_cache=False  # Phi-3 compatibility
            )
        
        response = knowledge_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()
    
    else:
        # Personality brain (Mistral + adapter)
        personality_model.set_adapter(domain)
        
        if memory_context:
            prompt = f"### Instruction:\n{memory_context}\n\nUser message: {query}\n\n### Response:\n"
        else:
            prompt = f"### Instruction:\n{query}\n\n### Response:\n"
        
        inputs = personality_tokenizer(prompt, return_tensors="pt").to(personality_model.device)
        
        with torch.no_grad():
            outputs = personality_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=personality_tokenizer.eos_token_id
            )
        
        response = personality_tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()
    
    # Clean response
    response = clean_response(response)
    
    # 4. Store interaction in memory
    if store_interaction:
        # Create summary of interaction
        interaction_summary = f"User asked about {domain}: {query[:50]}..."
        
        clara_memory.store(
            text=interaction_summary,
            memory_type="interaction",
            importance=0.4 + (conf * 0.4),  # Higher confidence = more important
            domain=domain if brain == "knowledge" else "personality",
            topic=domain.upper(),
            outcome="ANSWERED"
        )
    
    return response


def clara_remember(text: str, importance: float = 0.7, 
                   domain: str = "general") -> None:
    """
    Explicitly store something in Clara's memory
    
    Example:
        clara_remember("User prefers visual explanations", 
                      importance=0.9, domain="teaching")
    """
    clara_memory.store(
        text=text,
        memory_type="preference",
        importance=importance,
        domain=domain
    )
    print(f"✅ Stored: {text[:50]}...")


def clara_recall(query: str, top_k: int = 5) -> None:
    """
    Query Clara's memory directly
    """
    results = clara_memory.recall(query, top_k=top_k)
    
    print(f"\n🔍 Memory search: '{query}'")
    print("-" * 50)
    
    if not results:
        print("   No memories found")
        return
    
    for mem, score in results:
        print(f"   [{score:.2f}] [{mem.domain:10}] {mem.text[:50]}...")


def clara_stats() -> None:
    """Show Clara's memory statistics"""
    stats = clara_memory.stats()
    
    print("\n📊 Clara's Memory Stats")
    print("-" * 50)
    print(f"   Total memories: {stats['total_memories']}")
    print(f"   Symbols: {stats['symbols']}")
    print(f"   Size: {stats['size_kb']:.1f} KB")
    
    if stats['by_domain']:
        print("\n   By domain:")
        for domain, count in sorted(stats['by_domain'].items()):
            print(f"      {domain}: {count}")


def clara_save() -> None:
    """Save Clara's memory to disk"""
    clara_memory.save(MEMORY_FILE)


print("\n✅ Clara interface ready!")
print("\nAvailable functions:")
print("   clara(query)           - Talk to Clara")
print("   clara_remember(text)   - Store something in memory")
print("   clara_recall(query)    - Search Clara's memory")
print("   clara_stats()          - Show memory statistics")
print("   clara_save()           - Save memory to disk")

## Cell 8: Test Clara with Memory

In [ ]:
# ============================================================
# TEST CLARA WITH MEMORY
# ============================================================

print("=" * 60)
print("TESTING CLARA WITH HDC MEMORY")
print("=" * 60)

# Show initial memory state
clara_stats()

print("\n" + "-" * 60)
print("Test 1: Knowledge query (coding)")
print("-" * 60)
print("\n👤 User: How do I read a CSV file in Python?")
response = clara("How do I read a CSV file in Python?")
print(f"\n🤖 Clara: {response[:300]}...")

print("\n" + "-" * 60)
print("Test 2: Follow-up query (should use memory)")
print("-" * 60)
print("\n👤 User: What about writing to a CSV?")
response = clara("What about writing to a CSV?")
print(f"\n🤖 Clara: {response[:300]}...")

print("\n" + "-" * 60)
print("Test 3: Personality query")
print("-" * 60)
print("\n👤 User: I'm feeling stressed about my project deadline")
response = clara("I'm feeling stressed about my project deadline")
print(f"\n🤖 Clara: {response[:300]}...")

print("\n" + "-" * 60)
print("Test 4: Check memory")
print("-" * 60)
clara_recall("Python CSV")

## Cell 9: Interactive Session

In [ ]:
# ============================================================
# INTERACTIVE SESSION
# ============================================================

print("=" * 60)
print("INTERACTIVE CLARA SESSION")
print("=" * 60)
print("\nType your message to Clara.")
print("Special commands:")
print("   /memory     - Show memory stats")
print("   /recall X   - Search memory for X")
print("   /save       - Save memory to disk")
print("   /quit       - End session")
print("-" * 60)

while True:
    try:
        user_input = input("\n👤 You: ").strip()
        
        if not user_input:
            continue
        
        # Handle special commands
        if user_input.lower() == "/quit":
            print("\nEnding session. Don't forget to save memory!")
            break
        
        elif user_input.lower() == "/memory":
            clara_stats()
            continue
        
        elif user_input.lower().startswith("/recall "):
            query = user_input[8:].strip()
            clara_recall(query)
            continue
        
        elif user_input.lower() == "/save":
            clara_save()
            continue
        
        # Regular conversation
        response = clara(user_input)
        print(f"\n🤖 Clara: {response}")
        
    except KeyboardInterrupt:
        print("\n\nSession interrupted.")
        break

## Cell 10: Save Memory & Cleanup

In [ ]:
# ============================================================
# SAVE MEMORY & CLEANUP
# ============================================================

print("=" * 60)
print("SAVING & CLEANUP")
print("=" * 60)

# Final stats
clara_stats()

# Save memory
print("\n" + "-" * 60)
print("Saving memory...")
clara_save()

# Verify save
if os.path.exists(MEMORY_FILE):
    size = os.path.getsize(MEMORY_FILE) / 1024
    print(f"\n✅ Memory saved successfully!")
    print(f"   File: {MEMORY_FILE}")
    print(f"   Size: {size:.1f} KB")
else:
    print("\n❌ Failed to save memory")

print("\n" + "=" * 60)
print("SESSION COMPLETE")
print("=" * 60)
print("\nNext time you run this notebook, Clara will remember!")

---

## Optional: HDC Router Experiment

The cells below implement a pure HDC-based router as an alternative to the semantic router. 
This is experimental - use for comparison/research.

In [ ]:
# ============================================================
# EXPERIMENTAL: Pure HDC Router
# ============================================================

class HDCRouter:
    """
    Pure HDC-based routing (alternative to semantic router)
    
    Pros:
    - Fully interpretable (can explain via symbol overlap)
    - Compositional queries
    - Personal context integrated
    
    Cons:
    - Requires tuning symbol vocabulary
    - May need more data to match semantic accuracy
    """
    
    def __init__(self, memory: ClaraHDCMemory):
        self.memory = memory
        
        # Expert signatures as hypervectors
        print("Building expert signatures...")
        self.experts = {
            "medical": self._create_expert_signature([
                "symptoms", "disease", "treatment", "health", 
                "doctor", "medicine", "pain", "diagnosis"
            ]),
            "coding": self._create_expert_signature([
                "code", "programming", "python", "function",
                "debug", "error", "software", "algorithm"
            ]),
            "teaching": self._create_expert_signature([
                "explain", "learn", "understand", "concept",
                "tutorial", "beginner", "example", "how"
            ]),
            "quantum": self._create_expert_signature([
                "quantum", "qubit", "superposition", "entanglement",
                "particle", "physics", "wave", "measurement"
            ]),
            "personality": self._create_expert_signature([
                "feeling", "emotion", "stressed", "happy",
                "sad", "support", "friend", "worried"
            ]),
        }
        print("✅ HDC Router initialized")
    
    def _create_expert_signature(self, keywords: list) -> np.ndarray:
        """Create expert signature by bundling keyword symbols"""
        hvs = [self.memory._get_symbol(kw.upper()) for kw in keywords]
        return self.memory.bundle(hvs)
    
    def route(self, query: str, use_memory: bool = True) -> tuple:
        """
        Route using HDC similarity + memory context
        
        Returns: (brain, domain, confidence, explanation)
        """
        # Encode query
        query_hv = self.memory._text_to_hv(query)
        
        # Add personal context from memory
        if use_memory and len(self.memory.memories) > 0:
            memory_context = self.memory.get_context_for_routing(query)
            routing_hv = self.memory.bundle([query_hv, memory_context * 0.3])
        else:
            routing_hv = query_hv
        
        # Compare to expert signatures
        scores = {}
        for expert, sig_hv in self.experts.items():
            scores[expert] = self.memory.similarity(routing_hv, sig_hv)
        
        # Find best match
        best = max(scores, key=scores.get)
        brain = "personality" if best == "personality" else "knowledge"
        domain = "warmth" if best == "personality" else best
        
        # Explanation (interpretability!)
        explanation = {
            "all_scores": scores,
            "memory_used": use_memory and len(self.memory.memories) > 0
        }
        
        return brain, domain, scores[best], explanation


# Initialize HDC Router
print("\n" + "=" * 60)
print("HDC ROUTER (Experimental)")
print("=" * 60)

hdc_router = HDCRouter(clara_memory)

# Compare routing
print("\n📋 Comparing routers:")
print("-" * 70)
print(f"{'Query':<35} {'Semantic':<15} {'HDC':<15}")
print("-" * 70)

test_queries = [
    "How do I read a CSV in Python?",
    "I'm feeling anxious about tomorrow",
    "What is quantum entanglement?",
    "Explain how neural networks learn",
    "My chest hurts when I breathe",
]

for q in test_queries:
    sem_brain, sem_domain, sem_conf, _ = smart_route(q)
    hdc_brain, hdc_domain, hdc_conf, _ = hdc_router.route(q)
    
    sem_result = f"{sem_domain} ({sem_conf:.2f})"
    hdc_result = f"{hdc_domain} ({hdc_conf:.2f})"
    
    match = "✅" if sem_domain == hdc_domain else "❌"
    print(f"{q[:33]:<35} {sem_result:<15} {hdc_result:<15} {match}")